Submitted By: Group 1

`CAI, Edison`
`SUSADA, Stephanie Joy`


## FFT implementation in C using Cooley-Tukey Algorithm

In [63]:
%%writefile c_fft.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define PI 3.141592653589793238462643383279502884

typedef struct {
    double real;
    double imag;
} Complex;

// kernel

void fft(Complex * x, const unsigned int N){
    int i, j;

    if(N <= 1){
      return;
    }

    Complex* even = (Complex*)malloc(N / 2 * sizeof(Complex));
    Complex* odd = (Complex*)malloc(N / 2 * sizeof(Complex));

    for(i = 0; i < N/2; i++) {
      even[i] = x[2 * i];
      odd[i] = x[2 * i + 1];
      //if(N == 2)
        //printf("even = %lf, odd = %lf\n", even[i].real, odd[i].real);
    }

    fft(even, N/2);
    fft(odd, N/2);

    for (j = 0; j < N / 2; j++){
      double theta = -2 * PI * j / N;
      //printf("theta = %lf, i = %d, n = %d\n", theta,j, N);
      Complex w;
      Complex t;

      w.real = cos(theta);
      w.imag = sin(theta);
      //printf("theta = %lf,j = %d, wr = %lf, wi = %lf\n",theta,j,w.real,w.imag);

      t.real = w.real * odd[j].real - w.imag * odd[j].imag;
      t.imag = w.real * odd[j].imag + w.imag * odd[j].real;
        //printf("treal = %lf, oddr = %lf, oddi = %lf, wr = %lf, wi = %lf\n",t.real,odd[j].real,odd[j].imag, w.real, w.imag);
        //printf("x = %lf, e = %lf, t = %lf, j = %d", x[j].real, even[j].real,t.real,j);
      x[j].real = even[j].real + t.real;
      x[j].imag = even[j].imag + t.imag;
        //printf(", ans = %lf\n", x[j].real);
      x[j + N / 2].real = even[j].real - t.real;
      x[j + N / 2].imag = even[j].imag - t.imag;
    }

    free(even);
    free(odd);
}

int main(){

  int i, j;
  const unsigned int EXPO = 20;
  const unsigned int ARRAY_SIZE = pow(2,EXPO);
  const unsigned int ARRAY_BYTES = ARRAY_SIZE * sizeof(Complex);

  float *Y;
  Complex* input = (Complex*)malloc(ARRAY_BYTES);
  Y = (float*)malloc(ARRAY_BYTES);
  Complex* copy = (Complex*)malloc(ARRAY_BYTES);

  clock_t start, end;

  //initialize ARRAY
  srand(time(NULL));
  unsigned int error;

  for(i = 0; i < ARRAY_SIZE; i++){

    input[i].real = (double)rand() / RAND_MAX;
    //input[i].real = (double)i;
    copy[i].real = input[i].real;

    input[i].imag = 0.0;
    copy[i].imag = input[i].imag;
    //printf("%f\n", input[i].real); //for checking only
  }

    //fft(input, ARRAY_SIZE);
    double sum = 0;
    double time_taken;
    for(i = 0; i < 30; i++){
        start = clock();
        fft(input, ARRAY_SIZE);
        end = clock();
        time_taken = ((double)(end-start))*1e6/ CLOCKS_PER_SEC;
        sum += time_taken;

        fft(input, ARRAY_SIZE);
        fft(input, ARRAY_SIZE);
        fft(input, ARRAY_SIZE);

        double factor = pow(2, 2*EXPO);
        for (j = 0; j < ARRAY_SIZE; j++){
            if (!(abs(input[i].real/factor - copy[i].real) <= pow(10,-10) && abs(input[i].imag/factor - copy[i].imag) <= pow(10,-10))){
                error++;
                //printf("input = %.30lf, copy = %.30lf\n",(input[i].real/factor - copy[i].real), pow(10,-10));
            }
        }
        //printf("C Program finished with %u errors\n", error);

        /*
        printf("\nFFT Output:\n");
        for (j = 0; j < 10; j++) {
            printf("%.2lf + %.2lfi\n", input[j].real, input[j].imag);
        }*/

        for(j = 0; j < ARRAY_SIZE; j++){
            input[j].real = copy[j].real;
            input[j].imag = copy[j].imag;
        }
    }

    printf("C Program finished with %u errors\n", error);
    // Display FFT output
    /*
    printf("\nFFT Output:\n");
    for (i = 0; i < 10; i++) {
        printf("%.2lf + %.2lfi\n", input[i].real, input[i].imag);
    }*/
    double Ave = sum / 30;
    printf("\nC function takes average of %f us for array size %d \n", Ave, ARRAY_SIZE);

    //ERROR checking


    free(input);
    free(Y);

    return 0;
}

Overwriting c_fft.c


In [64]:
%%shell
g++ c_fft.c -o c_fft

In [ ]:
%%shell
./c_fft

# CUDA Implementation of FFT using Cooley-Tukey Algorithm

In [66]:
%%writefile cuda_fft.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define PI 3.141592653589793238462643383279502884

typedef struct {
    double real;
    double imag;
} Complex;

__global__
void dupe(Complex* x, Complex* y, const unsigned int N){
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    int count;

    for(count = i; count < N*2; count+=stride){
      y[i] = x[i];
    }
}

// kernel
__global__
void fft(Complex* x, Complex* y, int degree, const unsigned int N, const unsigned int exp){
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    int count;

    int m = exp - degree;

    for(count = i; count < N; count+=stride){
        int idx = (count * 2) - (count % (1 << (m-1)));
        int idy = idx + (1 << (m-1));
        double tempr = y[idx].real;
        double tempi = y[idx].imag;
        double tempor = y[idy].real;
        double tempoi = y[idy].imag;

        double theta = (-2 * PI * (idx/(1<<m))) / (1 << degree + 1);

        Complex w;
        Complex t;

        w.real = cos(theta);
        w.imag = sin(theta);

        t.real = w.real * tempor - w.imag * tempoi;
        t.imag = w.real * tempoi + w.imag * tempor;

        x[count].real = tempr + t.real;
        x[count].imag = tempi + t.imag;

        x[count + N].real = tempr - t.real;
        x[count + N].imag = tempi - t.imag;
        //printf("count = %d, count2 = %d, idx = %d, idy = %d, xr = %lf, tempr = %lf, tr = %lf, wr = %lf, wi = %lf, xidyr = %lf, xidyi = %lf\n", count, count+N, idx, idy, x[count].real, tempr, t.real, w.real, w.imag,x[idy].real, x[idy].imag);
    }
}

int main(){

  int i;
  const unsigned int EXPO = 20;
  const unsigned int ARRAY_SIZE = 1 << EXPO;
  const unsigned int ARRAY_BYTES = ARRAY_SIZE * sizeof(Complex);
  int numThreads = 1024;
  int numBlocks = (ARRAY_SIZE+numThreads-1) / numThreads;

  float *Y;
  Complex *input, *input2, *copy;
  cudaMallocManaged(&input, ARRAY_BYTES);
  cudaMallocManaged(&input2, ARRAY_BYTES);
  cudaMallocManaged(&Y, ARRAY_BYTES);
  cudaMallocManaged(&copy, ARRAY_BYTES);

  //initialize ARRAY
  srand(time(NULL));

  for(i = 0; i < ARRAY_SIZE; i++){

    //input[i].real = (double)rand() / RAND_MAX;
    input[i].real = (double)i;
    copy[i].real = input[i].real;

    input[i].imag = 0.0;
    copy[i].imag = input[i].imag;
    //printf("%f\n", input[i].real); //for checking only
  }


  for(i = 0; i < EXPO; i++){
    dupe <<<numBlocks, numThreads>>> (input, input2, ARRAY_SIZE);
    cudaDeviceSynchronize();
    fft <<<numBlocks, numThreads>>> (input, input2, i, ARRAY_SIZE/2, EXPO);
    cudaDeviceSynchronize();
  }

  for(i = 0; i < EXPO; i++){
    dupe <<<numBlocks, numThreads>>> (input, input2, ARRAY_SIZE);
    cudaDeviceSynchronize();
    fft <<<numBlocks, numThreads>>> (input, input2, i, ARRAY_SIZE/2, EXPO);
    cudaDeviceSynchronize();
  }
  for(i = 0; i < EXPO; i++){
    dupe <<<numBlocks, numThreads>>> (input, input2, ARRAY_SIZE);
    cudaDeviceSynchronize();
    fft <<<numBlocks, numThreads>>> (input, input2, i, ARRAY_SIZE/2, EXPO);
    cudaDeviceSynchronize();
  }
  for(i = 0; i < EXPO; i++){
    dupe <<<numBlocks, numThreads>>> (input, input2, ARRAY_SIZE);
    cudaDeviceSynchronize();
    fft <<<numBlocks, numThreads>>> (input, input2, i, ARRAY_SIZE/2, EXPO);
    cudaDeviceSynchronize();
  }

    // Display FFT output
    double factor = pow(2, 2*EXPO);
    printf("\nFFT Output:\n");
    for (i = 0; i < 10; i++) {
        printf("%.2lf + %.2lfi, %.2lf + %.2lfi\n", input[i].real, input[i].imag/factor, copy[i].real, copy[i].imag);
    }
    //ERROR checking

    unsigned int error;
    for (i = 0; i < ARRAY_SIZE; i++){
        if (!(abs(input[i].real/factor - copy[i].real) <= pow(10,-8) && abs(input[i].imag/factor - copy[i].imag) <= pow(10,-8))){
            error++;
            //printf("input = %.30lf, copy = %.30lf\n",(input[i].real/factor - copy[i].real), pow(10,-10));
        }
    }


    printf("CUDA Program finished with %u errors\n", error);

    cudaFree(input);
    cudaFree(input2);
    cudaFree(Y);
    cudaFree(copy);

    return 0;
}

Overwriting cuda_fft.cu


In [67]:
%%shell
nvcc cuda_fft.cu -o cuda_fft

In [ ]:
%%shell
nvprof ./cuda_fft